In [1]:
# Ejecucion de queries
from helper.helper import Helper

# Funciones adicionales
import sys
import os
sys.path.insert(1, '../utils')
from replica_n_meses.replica_n_meses import replica

In [2]:
helper = Helper(username='warboled', dsn='impala-prd')

general_params = {
    'proccess_zone': 'proceso',
    'proccess_area_zone': 'proceso_cap_analit_y_gob_de_inf',
    'pref': 'pmod'
}

path_etl = os.path.abspath(os.getcwd()).replace('notebooks', "etl")

2024-06-22 00:20:24 - [WARNING] - No se encontro la carpeta "c:\Users\warboled\REPOSITORIOS\prueba-modelos\notebooks\logs" para guardar los logs


 ___ __  __ ____   _    _        _    
|_ _|  \/  |  _ \ / \  | |      / \   
 | || |\/| | |_) / _ \ | |     / _ \  
 | || |  | |  __/ ___ \| |___ / ___ \ 
|___|_|  |_|_| /_/   \_\_____/_/   \_\
                                      
 _   _ _____ _     ____  _____ ____  
| | | | ____| |   |  _ \| ____|  _ \ 
| |_| |  _| | |   | |_) |  _| | |_) |
|  _  | |___| |___|  __/| |___|  _ < 
|_| |_|_____|_____|_|   |_____|_| \_\
                                     



## Replica Clientes

Crear base de abligaciones

In [13]:
helper.ejecutar_archivo(path_etl + "/0_obligaciones/0_obligaciones.sql", params=general_params)

-------------------------------------------------------------------------------------------------
                                       0_obligaciones.sql                                        
-------------------------------------------------------------------------------------------------
    i     tipo                    nombre                     estado     hora_inicio   duracion   
-------------------------------------------------------------------------------------------------
 196/198    DROP                proceso.pmod_obligaciones   finalizado   11:09:13 PM     00:00.1 
 197/198  CREATE                proceso.pmod_obligaciones   finalizado   11:09:14 PM     00:01.7 
 198/198 COMPUTE                proceso.pmod_obligaciones   finalizado   11:09:15 PM     00:01.2 
-------------------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
----

Replica n meses de las obligaciones para construcción de variables tendencia

In [10]:
params_replica = general_params
params_replica['base'] = 'pmod_obligaciones'
params_replica['fecha_corte'] = 'fecha_var_rpta_alt'
params_replica['vars_base'] = ",".join(["nit_enmascarado", "num_oblig_enmascarado", "num_oblig_orig_enmascarado"])
params_replica['path'] = os.path.abspath(os.getcwd()).replace('notebooks', "utils\\replica_n_meses")

In [11]:
replica(helper, 202308, 202401, params_replica)

Fechas del intervalo:  202208 202401
------------------------------------------------------------------------------
                              replica_fechas.sql                              
------------------------------------------------------------------------------
  i   tipo            nombre              estado     hora_inicio   duracion   
------------------------------------------------------------------------------
 4/9    DROP  proceso.pmod_fechas_anio   finalizado   11:04:14 PM     00:00.0 
 5/9  CREATE  proceso.pmod_fechas_anio   finalizado   11:04:15 PM     00:00.2 
 6/9 COMPUTE  proceso.pmod_fechas_anio   finalizado   11:04:15 PM     00:00.2 
 7/9    DROP   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.1 
 8/9  CREATE   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.2 
 9/9 COMPUTE   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.2 
------------------------------------------------------------------------------

Resumen de con

## Modulos ETL

In [5]:
helper.ejecutar_archivo(path_etl + "/0_obligaciones/1_obligaciones_cruce.sql", params=general_params)

2024-06-22 00:17:57 - [INFO] - Transcurrido: 1719033477, Tiempo de Refresco = 1000


------------------------------------------------------------------------------------
                              1_obligaciones_cruce.sql                              
------------------------------------------------------------------------------------
  i   tipo               nombre                 estado     hora_inicio   duracion   
------------------------------------------------------------------------------------
 1/3    DROP proceso.pmod_obligaciones_cruce   finalizado   12:17:57 AM     00:00.1 
 2/3  CREATE proceso.pmod_obligaciones_cruce   finalizado   12:17:57 AM     00:00.8 
 3/3 COMPUTE       proceso.pmod_obligaciones   finalizado   12:17:58 AM     00:01.4 
------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
---------------------------------------------
 1_obligaciones_cruce.sql      3     00:02.4 
-----------------------------

In [6]:
helper.ejecutar_archivo(path_etl + "/1_saldo_diario/0_acumulado_dia.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/1_saldo_diario/1_cruce_obligaciones_sd.sql", params=general_params)

--------------------------------------------------------------------------------------
                                 0_acumulado_dia.sql                                  
--------------------------------------------------------------------------------------
  i   tipo                nombre                  estado     hora_inicio   duracion   
--------------------------------------------------------------------------------------
 4/6    DROP proceso.pmod_saldo_diario_add_dia   finalizado   12:18:00 AM     00:00.1 
 5/6  CREATE proceso.pmod_saldo_diario_add_dia   finalizado   12:18:00 AM     00:07.3 
 6/6 COMPUTE proceso.pmod_saldo_diario_add_dia   finalizado   12:18:07 AM     00:00.6 
--------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
---------------------------------------------
 1_obligaciones_cruce.sql      3     00:02.4 
      0_acu

In [7]:
helper.ejecutar_archivo(path_etl + "/2_maestro_cuotas/0_filtro_maestro.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/2_maestro_cuotas/1_cruce_obligaciones_mc.sql", params=general_params)

-------------------------------------------------------------------------------------------
                                   0_filtro_maestro.sql                                    
-------------------------------------------------------------------------------------------
   i    tipo                  nombre                   estado     hora_inicio   duracion   
-------------------------------------------------------------------------------------------
 10/12    DROP               proceso.maestro_cuotas   finalizado   12:18:11 AM     00:00.1 
 11/12  CREATE               proceso.maestro_cuotas   finalizado   12:18:11 AM     00:01.1 
 12/12 COMPUTE               proceso.maestro_cuotas   finalizado   12:18:12 AM     00:01.1 
-------------------------------------------------------------------------------------------

Resumen de consultas:
------------------------------------------------
         documento           total    duracion 
------------------------------------------------
   

In [8]:
helper.ejecutar_archivo(path_etl + "/3_mcd/0_filtro_mcd.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/3_mcd/1_cruce_obligaciones_mcd.sql", params=general_params)

-----------------------------------------------------------------------------------------------
                                       0_filtro_mcd.sql                                        
-----------------------------------------------------------------------------------------------
   i    tipo                    nombre                     estado     hora_inicio   duracion   
-----------------------------------------------------------------------------------------------
 16/18    DROP                         proceso.pmod_mcd   finalizado   12:18:19 AM     00:00.2 
 17/18  CREATE                         proceso.pmod_mcd   finalizado   12:18:19 AM     00:03.0 
 18/18 COMPUTE                         proceso.pmod_mcd   finalizado   12:18:22 AM     00:02.6 
-----------------------------------------------------------------------------------------------

Resumen de consultas:
------------------------------------------------
         documento           total    duracion 
----------------

In [5]:
helper.ejecutar_archivo(path_etl + "/99_base_modelo/0_base_cruce.sql", params=general_params)

----------------------------------------------------------------------------
                              0_base_cruce.sql                              
----------------------------------------------------------------------------
  i   tipo           nombre             estado     hora_inicio   duracion   
----------------------------------------------------------------------------
 6/8    DROP proceso.pmod_base_cruce   finalizado   12:25:56 AM     00:00.1 
 7/8  CREATE proceso.pmod_base_cruce   finalizado   12:25:56 AM     00:02.5 
 8/8 COMPUTE proceso.pmod_base_cruce   finalizado   12:25:58 AM     00:00.7 
----------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------
   documento      total    duracion 
-------------------------------------
 0_base_cruce.sql      8     00:13.8 
-------------------------------------



## DESCARGA BASE

In [7]:
download_query = "SELECT * FROM {proccess_zone}.pmod_base_cruce"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

--------------------------------------------------------------------------------
   i     tipo            nombre             estado     hora_inicio   duracion   
--------------------------------------------------------------------------------
 10/10 DATAFRAME                         convirtiendo   12:29:10 AM             

2024-06-22 00:29:38 - [INFO] - 680,800 filas, 26 columnas, 00:00.4 consultando, 00:25.7 descargando, 00:01.7 convirtiendo


 10/10 DATAFRAME                           finalizado   12:29:10 AM     00:28.0 
--------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,sd_avg_dia_sld_cap_final,sd_max_dia_sld_cap_final,sd_min_sld_cap_final,sd_stddevpop_dia_sld_cap_final,sd_max_dia_nueva_altura_mora,sd_min_dia_nueva_altura_mora,...,sd_min_dia_vlr_vencido,sd_stddevpop_dia_vlr_vencido,mc_valor_cuota_mes,mc_pago_total,mc_porc_pago,mcd_total_ing,mcd_tot_activos,mcd_egresos_mes,mcd_tot_patrimonio,var_rpta_alt
0,631971,446033,634309,202401,8.676510e+07,8.676510e+07,8.676510e+07,0.000000,51.0,22.0,...,2846031.0,1.629924e+06,None,None,None,31315420.00,108282000.00,650000.00,108282000.00,NaN
1,150236,16858,1063484,202308,1.204220e+07,1.204721e+07,1.189194e+07,27433.412234,33.0,4.0,...,311111.0,1.081878e+05,None,None,None,2500000.00,2000000.00,300000.00,2000000.00,1.0
2,382084,469034,611308,202308,2.362085e+07,2.374322e+07,2.353247e+07,103993.351588,0.0,0.0,...,0.0,0.000000e+00,None,None,None,3004000.00,34444000.00,1200000.00,34444000.00,0.0
3,176011,609245,471097,202312,1.779510e+08,1.780275e+08,1.777406e+08,126858.850768,51.0,24.0,...,1811160.0,8.334244e+05,None,None,None,6948168.00,225515000.00,100000.00,225515000.00,1.0
4,214474,959259,128592,202312,3.246340e+06,3.294020e+06,3.115220e+06,79068.335002,15.0,0.0,...,161258.0,4.613921e+04,None,None,None,15800000.00,230000000.00,500000.00,203925239.00,1.0


In [8]:
data.to_csv('../data/raw/pdmod_base_model.csv', index=False)